In [1]:
#SENDING PLATFORMS API SSO INTEGRATIONS TO SNOWFLAKE 

In [2]:
#IMPORT LIBRARIES

import pymysql as mysql
import pandas as pd
import psycopg2 as pg2


In [3]:
pip install pymssql


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymssql

conn = pymssql.connect(
         server="eloomi-mssql-server-prod.database.windows.net",
         port=1433,
         user="emir",
         password="jAU9qHUz9MuK2ejTern5",
         database="eloomi-mssql-db-global-platforms-prod")
conn
cursor = conn.cursor()
cursor.execute("SELECT * FROM platform_domains")

In [5]:
db_infinite_cursor = conn.cursor()  


In [6]:
db_infinite_cursor.execute('''
      select p.platform_id,
       p.name,
       count(a.id)                                   api_tokens,
       iif(p.sso_active = 1, 'true', 'false')        sso_enabled,
       iif(count(i.created_at) > 0, 'true', 'false') native_integrations_enabled,
       iif(count(pd.id) > 0, 'true', 'false')        custom_domain_enabled
from platform p
         left join api_tokens a on p.platform_id = a.platform_id and a.deleted_at is null
         left join integrations i
                   on p.platform_id = i.platform_id and i.partner_id = 1 and i.disabled_at is null
         left join platform_domains pd
                   on p.platform_id = pd.platform_id and pd.type = 'customer_domain' and pd.deleted_at is null
where p.active = 1
group by p.platform_id, p.name, p.sso_active;
                  ''')


In [7]:
db_infinite_platform_id = db_infinite_cursor.fetchall()
cols = []

for i in db_infinite_cursor.description:
    cols.append(i[0])

db_infinite_platform_id = pd.DataFrame(db_infinite_platform_id, columns = cols)  

In [8]:
db_infinite_platform_id.columns

Index(['platform_id', 'name', 'api_tokens', 'sso_enabled',
       'native_integrations_enabled', 'custom_domain_enabled'],
      dtype='object')

In [9]:
db_platform_api_sso = db_infinite_platform_id

In [10]:
db_platform_api_sso = db_platform_api_sso.rename(columns={
    'api_tokens': 'API Tokens Enabled',
    'sso_enabled': 'SSO Enabled',
    'native_integrations_enabled': 'Native HCM Integration Enabled',
    'custom_domain_enabled': 'Custom Domain Enabled'
})

In [11]:
db_platform_api_sso.head(5)

,platform_id,name,API Tokens Enabled,SSO Enabled,Native HCM Integration Enabled,Custom Domain Enabled
0,1177,External,28,true,true,false
1,1600,thomas,0,false,false,false
2,1623,Cornerstoneak,0,false,false,false
3,1737,sti,0,false,false,false
4,1745,barma,6,false,false,true


In [14]:
import pandas as pd
import snowflake.connector

def send_df_to_snowflake(df, table_name, schema, database, user, password, account, warehouse, role):
    """
    Sends a DataFrame to Snowflake and replaces the existing table.
    
    Parameters:
    - df: pandas DataFrame to be sent to Snowflake.
    - table_name: Name of the table in Snowflake.
    - schema: Schema of the table in Snowflake.
    - database: Database of the table in Snowflake.
    - user: Snowflake user.
    - password: Snowflake password.
    - account: Snowflake account identifier.
    - warehouse: Snowflake warehouse to use.
    - role: Snowflake role to use.
    """
    
    # Establish Snowflake connection
    conn = snowflake.connector.connect(
        user=user,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema,
        role =role
    )
    
    # Create a cursor object
    cur = conn.cursor()
    
    temp_csv = 'temp_file.csv'
    df.to_csv(temp_csv, index=False)
    
    # Use Snowflake PUT command to upload the CSV file to a Snowflake stage
    cur.execute(f"PUT file://{temp_csv} @%{table_name}")
    
    # Delete existing table data
    cur.execute(f"TRUNCATE TABLE {schema}.{table_name}")
    
    # Load data from the staged CSV file into the Snowflake table
    cur.execute(f"""
        COPY INTO {schema}.{table_name}
        FROM @%{table_name}/{temp_csv.split('/')[-1]}
        FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"')
        ON_ERROR = 'CONTINUE'
    """)
    
    # Remove the staged file
    cur.execute(f"REMOVE @%{table_name}/{temp_csv.split('/')[-1]}")
    
    # Close the cursor and connection
    cur.close()
    conn.close()

In [15]:
# Send DataFrame to Snowflake
send_df_to_snowflake(
        df=db_platform_api_sso,
        table_name='PLATFORM_API_SSO',
        schema='PLATFORM_DATA',
        database='RAW_DB',
        user='emirkabasoglu',
        password='7Bb13287',
        account='se63477.north-europe.azure',
        warehouse='TRANSFORMING_WH',
        role ='ACCOUNTADMIN'
    )

In [ ]:
# OUTPUT FINAL FILE WITH ALL DIFFERENCES
db_platform_api_sso.to_excel('/Users/emirkabasoglu/Desktop/Eloomi/VS Codes/Jupyter Notebook Scripts/Snowflake Scripts/snowflake-to-excel.xlsx')

In [ ]:
#CREATING THE SNOWFLAKE CONNECTION

In [ ]:
#IMPORT LIBRARIES
#import snowflake.connector
#from snowflake.connector.pandas_tools import write_pandas
#import os
#import pandas as pd
#from sqlalchemy import create_engine



In [ ]:

# Snowflake connection parameters
#SNOWFLAKE_USER = 'emirkabasoglu'
#SNOWFLAKE_PASSWORD = '7Bb13287'
#SNOWFLAKE_ACCOUNT = 'se63477.north-europe.azure'
#SNOWFLAKE_WAREHOUSE = 'LOADING_WH'
#SNOWFLAKE_DATABASE = 'RAW_DB'
#SNOWFLAKE_SCHEMA = 'PLATFORM_DATA'

# Establish a connection to Snowflake
#conn = snowflake.connector.connect(
    #user=SNOWFLAKE_USER,
    #password=SNOWFLAKE_PASSWORD,
    #account=SNOWFLAKE_ACCOUNT,
    #warehouse=SNOWFLAKE_WAREHOUSE,
    #database=SNOWFLAKE_DATABASE,
    #schema=SNOWFLAKE_SCHEMA
#)


In [ ]:
#CREATE THE ENGINE THAT CONNECTS TO THE DATABASE
#from sqlalchemy import create_engine

#engine = create_engine(f'snowflake://{SNOWFLAKE_USER}:{SNOWFLAKE_PASSWORD}@{SNOWFLAKE_ACCOUNT}/{SNOWFLAKE_DATABASE}/{SNOWFLAKE_SCHEMA}?warehouse={SNOWFLAKE_WAREHOUSE}'
#)